In [6]:
import pandas as pd
import requests
import os

os.chdir('/mnt/c/Users/hijaz tr/Desktop/cadCADProject1/tokenspice')

In [2]:

def fetch_pool_data(pool_id= "0x4e68ccd3e89f51c3074ca5072bbac773960dfa36", UNISWAP_V3_SUBGRAPH_URL = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'):
    # Fetching liquidity positions
    liquidity_query = """
    {
      positions(where: { pool: "%s" ,liquidity_gt: "0"}) {
        liquidity
        tickLower {
          price0
          tickIdx
        }
        tickUpper {
          price0
          tickIdx
        }
      }
    }
    """ % pool_id
    liquidity_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': liquidity_query}).json()

    tick_query="""
     {
     ticks(where: {liquidityGross_gt: "0", liquidityNet_gt: "0", pool: "%s"}) {
     
      tickIdx
      liquidityGross
      liquidityNet
      
    }
  }
  """% pool_id
    tick_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': tick_query}).json()
    # Fetching pool price, volume, fees, and reserves
    global_query = """
    {
      pools(where: { id: "%s" }) {
        token0Price
        token1Price
        volumeUSD
        feesUSD
        totalValueLockedToken0
        totalValueLockedToken1
      }
    }
    """ % pool_id
    global_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': global_query}).json()

    # Fetching swaps and trades
    swaps_query = """
    {
      swaps(where: { pool: "%s" }, first: 10, orderBy: timestamp, orderDirection: desc) {
        amount0
        amount1
        amountUSD
        timestamp
      }
    }
    """ % pool_id
    swaps_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': swaps_query}).json()

        # Constructing the state representation with float data type
    state = {
        'positions': [(float(position['tickLower']['tickIdx']), float(position['tickUpper']['tickIdx']), float(position['liquidity'])) for position in liquidity_data['data']['positions']],
        'pool_price': float(global_data['data']['pools'][0]['token1Price']),
        'ticks': [(float(tick['tickIdx']), float(tick['liquidityGross']), float(tick['liquidityNet'])) for tick in tick_data['data']['ticks']]
    }

    return state
# get events data

In [3]:
state=fetch_pool_data()


In [4]:
df_positions=pd.DataFrame(state['positions'])
df_positions.head()

,0,1,2
0,-194400.0,-193320.0,1.143324e+12
1,-196260.0,-191160.0,2.149580e+13
2,-195120.0,-194520.0,3.471954e+13
3,-194700.0,-194580.0,4.765494e+12
4,-194580.0,-184200.0,1.471111e+13


In [7]:
from util.globaltokens import weth_usdc_pool
from util.constants import GOD_ACCOUNT
from util.base18 import toBase18, price_to_valid_tick
env=weth_usdc_pool
print(env.get_lp_all_positions(GOD_ACCOUNT))

NameError: name 'functoolz' is not defined

In [8]:
global_state=env.get_global_state()
global_state_df=pd.DataFrame([global_state])
global_state_df

,curr_sqrtPriceX96,curr_price,tick,feeGrowthGlobal0X128,feeGrowthGlobal1X128,liquidity_raw,blockTimestamp,tickCumulative,secondsPerLiquidityCumulativeX128
0,3541946836890595876327551269414,1998.595518,76005,8043322633717853653763433794043,0,380755745076068400000,1697394631,34712170838,408135170782626187638558


In [9]:
tick_state=env.get_pool_state_for_all_ticks(1000,3000)
tick_state_df = pd.DataFrame.from_dict(tick_state, orient='index')
tick_state_df

,tick,liquidityGross_raw,liquidityNet_raw,feeGrowthOutside0X128,feeGrowthOutside1X128,tickCumulativeOutside,tickBitmap
74400,74400,0,0,0,0,0,0
69060,69060,380755745076068400000,380755745076068400000,0,0,0,0
78660,78660,21470565349021557000000,21470565349021557000000,0,0,0,0
80040,80040,380755745076068400000,-380755745076068400000,0,0,0,0
79020,79020,21470565349021557000000,-21470565349021557000000,0,0,0,0
76980,76980,0,0,0,0,0,0
77400,77400,0,0,0,0,0,0
74940,74940,0,0,0,0,0,0


In [10]:
positions_data=env.get_pool_state_for_all_positions()
positions_df = pd.DataFrame.from_dict(positions_data, orient='index')
positions_df

,position_key,liquidity_provider,tick_lower,tick_upper,_liquidity_raw,feeGrowthInside0LastX128,feeGrowthInside1LastX128,tokensOwed0_raw,tokensOwed1_raw
0x330997E70b83f1a562490FCaA5996314fA5a971a_74940_76980,0x330997E70b83f1a562490FCaA5996314fA5a971a_749...,0x330997E70b83f1a562490FCaA5996314fA5a971a,74940,76980,0,0,0,0,0
0x330997E70b83f1a562490FCaA5996314fA5a971a_74400_77400,0x330997E70b83f1a562490FCaA5996314fA5a971a_744...,0x330997E70b83f1a562490FCaA5996314fA5a971a,74400,77400,0,0,0,0,0
0x330997E70b83f1a562490FCaA5996314fA5a971a_69060_80040,0x330997E70b83f1a562490FCaA5996314fA5a971a_690...,0x330997E70b83f1a562490FCaA5996314fA5a971a,69060,80040,380755745076068400000,0,0,0,0
0x330997E70b83f1a562490FCaA5996314fA5a971a_7600_8100,0x330997E70b83f1a562490FCaA5996314fA5a971a_760...,0x330997E70b83f1a562490FCaA5996314fA5a971a,7600,8100,0,0,0,0,0
0x330997E70b83f1a562490FCaA5996314fA5a971a_78660_79020,0x330997E70b83f1a562490FCaA5996314fA5a971a_786...,0x330997E70b83f1a562490FCaA5996314fA5a971a,78660,79020,21470565349021557000000,0,0,0,0


# integrate netlists here

In [14]:
sim_strategy = SimStrategy()


sim_state = SimState(ss=sim_strategy)


from engine.SimEngine import SimEngine

output_dir = "model_outdir_csv"
netlist_log_func = netlist_createLogData

engine = SimEngine(sim_state, output_dir, netlist_log_func)
engine.run()

mnemonic: 'child knock end approve blade apple pottery enrich message agree waste rug'
mnemonic: 'soup find flee inmate foot price piano maze city lyrics force electric'
{'Mint': [OrderedDict([('sender', '0xd077A14F781E00409c1c3c9a3eBa8d27b3104ade'), ('owner', '0xd077A14F781E00409c1c3c9a3eBa8d27b3104ade'), ('tickLower', 74040), ('tickUpper', 77820), ('amount', 359567405915170100000), ('amount0', 694609467094794631), ('amount1', 1510393888265772888615)])]}
{'Transfer': [OrderedDict([('from', '0x545C12B4504a026F9D4D8828C707F847821F4673'), ('to', '0x1A052fDa16246AdeDEF8fF09E308efFc0F3ab63b'), ('value', 129274223057914537)])], 'Swap': [OrderedDict([('sender', '0x1A052fDa16246AdeDEF8fF09E308efFc0F3ab63b'), ('recipient', '0x1A052fDa16246AdeDEF8fF09E308efFc0F3ab63b'), ('amount0', -129274223057914537), ('amount1', 259326498808711380992), ('sqrtPriceX96', 3543191670733485921977148586851), ('liquidity', 380758448079696334188907000), ('tick', 76012)])]}
{'Burn': [OrderedDict([('owner', '0xd077A14